In [3]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd

test_df = pd.read_csv("17_test.csv")

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import Audio
import soundfile as sf
import torch
import os

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained(
    "classla/wav2vec2-xls-r-parlaspeech-hr")
model = Wav2Vec2ForCTC.from_pretrained("classla/wav2vec2-xls-r-parlaspeech-hr")

test_df["audio"] = test_df.hashname.apply(lambda s: "transfer_10/"+s)
import datasets
from datasets import load_dataset, load_metric, Audio



def get_transcript(audio_filepath:str):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model.to(device)(input_values.to(device)).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription
test_df["predicted"] = test_df.audio.apply(get_transcript)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
audio_filepath=test_df.audio[0]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
speech, sample_rate = sf.read(audio_filepath)
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)

logits = model.to(device)(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0]).lower()
transcription

'ribarsku brodicu i sada što se događa imamo sustav enormnog licemjerja licemjerja u kojem bankar iz hbora koji je odobrio kredit od jedanaestpet milijuna kuna eph holdingu neće snositi nikakvu odgovornost'

In [5]:
cer = cer_metric.compute(references = test_df.human_transcript,
                 predictions = test_df.predicted
                )
cer

0.054202649545598405

In [6]:
wer = wer_metric.compute(references = test_df.human_transcript,
                 predictions = test_df.predicted
                )
wer

0.13936537513015837